In [1]:
using Pkg
Pkg.activate("..")

using MLJFair, MLJ

 Activating environment at `~/Desktop/MLJFair/Project.toml`


### Loading toy data

In [2]:
X, y, ŷ = @load_toydata;

In [3]:
X

,Sex,Ethnicity,Highest degree,Job type
,Categorical…,Categorical…,Categorical…,Categorical…
1,M,Native,H. school,Board
2,M,Native,Univ.,Board
3,M,Native,H. school,Board
4,M,Non-nat.,H. school,Healthcare
5,M,Non-nat.,Univ.,Healthcare
6,F,Non-nat.,Univ.,Education
7,F,Native,H. school,Education
8,F,Native,None,Healthcare
9,F,Non-nat.,Univ.,Education


In [4]:
y, ŷ

(CategoricalArrays.CategoricalValue{Int64,UInt32}[1, 1, 1, 1, 0, 0, 0, 1, 0, 1], CategoricalArrays.CategoricalValue{Int64,UInt32}[1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

# Loading the Classifier and Wrapping it

## ReweighingSampling Algorithm

In [5]:
model = ConstantClassifier()

wrappedModel = ReweighingSamplingWrapper(model; grp=:Sex)

ReweighingSamplingWrapper(
    grp = :Sex,
    classifier = ConstantClassifier()) @ 1…19

### Using MLJ for packaging data with the Machine Learning model (Wrapped by fairness algorithm)

In [6]:
mach = machine(wrappedModel, X, y)

Machine{ReweighingSamplingWrapper} @ 4…53


### Fitting the Wrapped Model on the packaged data.

Everything will be handled underneath the hood. The reweighing algorithm will be fitted first to generate weights for all comninations of classes and outcomes. Then these weights are used to sample the training data and then classifier is trained....**And all this is done automatically by MLJFair. You simply have to choose the wrapper of your choice!!**

In [7]:
fit!(mach)

┌ Info: Training Machine{ReweighingSamplingWrapper} @ 4…53.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/O5b6j/src/machines.jl:187


Machine{ReweighingSamplingWrapper} @ 4…53


Prediction is also handled underneath the hood for all sorts of algorithms.

MLJFair leverages MLJ to make it really easy to do fitting and prediction.
Internals: MLJ creates a node for the end classifier and automatically calls predict on that node. So I didn't have to eplicitly code a predict function

In [8]:
predict(mach, X[1:5, :])

5-element Array{UnivariateFinite{Int64,UInt32,Float64},1}:
 UnivariateFinite(0=>0.5, 1=>0.5)
 UnivariateFinite(0=>0.5, 1=>0.5)
 UnivariateFinite(0=>0.5, 1=>0.5)
 UnivariateFinite(0=>0.5, 1=>0.5)
 UnivariateFinite(0=>0.5, 1=>0.5)

## Equalized Odds Postprocessing Algorithm

Again, you have to simply load the classifier model and wrap it with fairness algorithm of your choice!

In [9]:
model2 = ConstantClassifier()

wrappedModel2 = EqOddsWrapper(model2; grp=:Sex)

EqOddsWrapper(
    grp = :Sex,
    classifier = ConstantClassifier()) @ 2…99

### Package data with wrapped model  and then fit the model on that data

In [10]:
mach2 = machine(wrappedModel2, X, y)

fit!(mach2)

┌ Info: Training Machine{EqOddsWrapper} @ 1…95.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/O5b6j/src/machines.jl:187
┌ Info: Training Machine{ConstantClassifier} @ 4…83.
└ @ MLJBase /home/ashryaagr/.julia/packages/MLJBase/O5b6j/src/machines.jl:187


Machine{EqOddsWrapper} @ 1…95


### You can also see the training results i.e. the fitted params

What you shall see as output below is the fitting result for:
- Equalized Odds : It is in form of **[sp2n, sn2p, op2n, on2p]**. s denotes priviledged group and p2n denoted conversion from Positive to Negative
- Constant Classifier

In [11]:
mach2.fitresult

2-element Array{Any,1}:
 [0.0, 1.0, 0.0, 1.0]
 UnivariateFinite(0=>0.4, 1=>0.6)

### Predict

In [12]:
predict(mach2, X[5:10, :])

6-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1